In [ ]:
!mkdir docker

In [ ]:
%%writefile Dockerfile

FROM python:3.9-slim-buster

RUN pip install pandas==2.2.2 scikit-learn==1.4.2 prophet==1.1.5

COPY code/preprocess.py /opt/ml/code/preprocess.py
COPY code/train.py /opt/ml/code/train.py

ENV SAGEMAKER_PROGRAM preprocess.py
ENV SAGEMAKER_PROGRAM train.py
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

In [ ]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ecr_repository = 'demo_prophet_promotional_forecasting'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

In [ ]:
ecr_repository = 'demo_prophet_promotional_forecasting'

In [ ]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository docker

In [ ]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com

In [ ]:
aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin account_id.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!aws ecr create-repository --repository-name $ecr_repository

In [ ]:
!docker tag {ecr_repository + tag} $processing_repository_uri

In [ ]:
!docker push $processing_repository_uri